In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [3]:
pip install openpyxl

# EDA

 ## Importing Data 

In [4]:
df_train = pd.read_excel('/kaggle/input/flight-fare-prediction-mh/Data_Train.xlsx')
df_train.head()

In [5]:
df_test = pd.read_excel('/kaggle/input/flight-fare-prediction-mh/Sample_submission.xlsx')
df_test.head()

In [6]:
print(df_train.shape)
print(df_test.shape)

In [7]:
df_train.info()

In [8]:
df_train.describe()

## Finding the Null values



In [9]:
df_train.isnull().sum()

# Data Cleaning

In [10]:
#import missingno as msno
#msno.bar(df_train)
#plt.show

import missingno as msno
msno.heatmap(df_train)

In [11]:
df_train.columns

## Dropping the Null Values


In [12]:
df_train.dropna(inplace=True)

In [13]:
df_train.isnull().sum()

In [14]:
df_train

In [15]:
df_train.dtypes

In [16]:
def change_into_datetime(col):
    df_train[col]=pd.to_datetime(df_train[col])

In [17]:
df_train.columns

In [18]:
for i in ['Date_of_Journey','Dep_Time', 'Arrival_Time']:
    change_into_datetime(i)

In [19]:
df_train.dtypes

In [20]:
df_train.value_counts()

In [21]:
df_train['journey_day']=df_train['Date_of_Journey'].dt.day
df_train['journey_month']=df_train['Date_of_Journey'].dt.month

In [22]:
df_train.head()

In [23]:
df_train.drop('Date_of_Journey', axis=1, inplace=True)
df_train.head()

# Data Visualization

### First of all lets check the count of the airlines in the visual graph

In [24]:
plot=plt.figure()
sns.countplot('Airline',data=df_train)
plt.xticks(rotation=90)

In [25]:
#plot showing Airline with respect to Price
sns.catplot(y = "Price", x = "Airline", data = df_train.sort_values("Price", ascending = False), kind="boxen", height = 6, aspect = 3)
plt.show()

In [26]:
#Source vs Price
sns.catplot(y = "Price", x = "Source", data = df_train.sort_values("Price", ascending = False), kind="boxen", height = 4, aspect = 3)
plt.show()

### Keeping the Source, Destination and Duration in mind enhancing a column visualization with 3D view

In [27]:
!pip install plotly matplotlib seaborn --quiet
import plotly.express as px

fig = px.scatter_3d(df_train, x='Source', y='Destination', z='Duration')
fig.update_traces(marker_size=3, marker_opacity=0.5)
fig.show()

In [28]:
#Extracting Time related data

def extract_hour(data,col):
    data[col+'_hour']=data[col].dt.hour
    
def extract_min(data,col):
    data[col+'_min']=data[col].dt.minute
    
def drop_col(data,col):
    data.drop(col,axis=1,inplace=True)



In [29]:
extract_hour(df_train,'Dep_Time')
extract_min(df_train,'Dep_Time')

In [30]:
extract_hour(df_train,'Arrival_Time')
extract_min(df_train,'Arrival_Time')

In [31]:
df_train.drop(['Dep_Time','Arrival_Time'], axis=1, inplace=True)
df_train.head()

In [32]:
duration=list(df_train['Duration'])
for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        pass
    else:
        if 'h' in duration[i]: # Check if duration contains only hour
             duration[i]=duration[i] + ' 0m' # Adds 0 minute
        else:
            duration[i]='0h '+ duration[i]

In [33]:
df_train['Duration']=duration
df_train.head()

In [34]:
def hour(x):
    return x.split(' ')[0][0:-1]

def minutes(x):
    return x.split(' ')[1][0:-1]

In [35]:
df_train['dur_hour']=df_train['Duration'].apply(hour)
df_train['dur_min']=df_train['Duration'].apply(minutes)

In [36]:
df_train.head()

In [37]:
df_train.dtypes

In [38]:
df_train['dur_hour'] = df_train['dur_hour'].astype(int)
df_train['dur_min'] = df_train['dur_min'].astype(int)

In [39]:
df_train.dtypes

In [40]:
#Parent Airline v/s the Price Valuation plot
fig = px.histogram(df_train, 
                   x='Price', 
                   marginal='box', 
                   color='Airline', 
                   color_discrete_sequence=['green', 'grey','red','orange','blue','blueviolet', 'brown', 'burlywood', 'chocolate', 'darkblue','olive','pink'], 
                   title='Parent Airline v/s Price Valuation')
fig.update_layout(bargap=0.55)
fig.show()

In [41]:
#Total Stops vs Price visualization with scatter plot
fig = px.scatter(df_train, 
                 x='Total_Stops', 
                 y='Price', 
                 color='Source', 
                 opacity=0.8, 
                 hover_data=['Airline'], 
                 title='Total_Stops vs. Price')
fig.update_traces(marker_size=10)
fig.show()

In [42]:
#prevously we did Source, Destination and Duration 
#Now lets checkout aa 3D View of Source, Destination and Price

fig = px.scatter_3d(df_train, x='Source', y='Destination', z='Price')
fig.update_traces(marker_size=3, marker_opacity=0.5)
fig.show()

In [43]:
df_train['Route'].value_counts()

In [44]:
df_train.Additional_Info.value_counts()

In [45]:
df_train.drop(['Route','Additional_Info'], axis=1, inplace=True)
df_train.head(2)

In [46]:
df_train.nunique()

In [47]:
df_train.groupby('Source').sum()

In [48]:
df_train.dtypes

# Model Training

In [49]:
model_df = df_train.loc[:,["Airline","Source","Destination","Total_Stops","journey_day","journey_month",
                     "Dep_Time_hour","Dep_Time_min","Arrival_Time_hour","Arrival_Time_min","Price"]]
model_df

In [50]:
#Defining X and y

X = model_df.drop(columns=['Price']) #input
y = model_df['Price'] #target 

In [51]:
X.head()

In [52]:
y.head()

### Identifying Numerical and Categorical columns

In [53]:
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols

In [54]:
categorical_cols = X.select_dtypes('object').columns.tolist()
categorical_cols

### Features Transformation and Scaling - MinMaxScaler, OneHotEncoding
Scale Numeric values

In [55]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(model_df[numeric_cols])

In [56]:
print('Minimum:')
list(scaler.data_min_)

In [57]:
print('Maximum:')
list(scaler.data_max_)

In [58]:
model_df[numeric_cols].describe().loc[['min', 'max']]

In [59]:
X[numeric_cols] = scaler.transform(X[numeric_cols])
X[numeric_cols].describe()

In [60]:
model_df[categorical_cols].nunique()

### Encode Categorical columns

In [61]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(model_df[categorical_cols])

In [62]:
encoder.categories_

In [63]:
encoded_cols = list(encoder.get_feature_names(categorical_cols))
print(encoded_cols)

In [64]:
X.isna().value_counts()

In [65]:
X[encoded_cols] = encoder.transform(X[categorical_cols])

In [66]:
X_train = X[numeric_cols + encoded_cols]
X_test= X[numeric_cols + encoded_cols]

In [67]:
## Train-Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [68]:
print('X_train.shape :', X_train.shape)
print('X_test.shape :', X_test.shape)
print('y_train.shape :', y_train.shape)
print('y_test.shape :', y_test.shape)

In [69]:
model_df.corr()

In [70]:
fig, ax = plt.subplots(figsize=(10,7)) 
sns.heatmap(model_df.corr(), cmap='Blues', annot=True)
plt.title('Correlation Matrix');

#### Regression Model Building and Performance Evaluation

In [71]:
X_train = X_train[numeric_cols + encoded_cols]
X_test = X_test[numeric_cols + encoded_cols]

In [72]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
def predict(ml_model):

    # Defining model
    print('Model: {}'.format(ml_model))

    # model fit with data
    model= ml_model.fit(X_train,y_train)

    # Model Training Score
    print("Training score: {}".format(model.score(X_train,y_train)))

    # Model Predictions
    predictions = model.predict(X_test)

    # Define r2 score for Regressor Model
    r2score=r2_score(y_test,predictions) 
    print("r2 score: {}".format(r2score))
    
     # Model Evoluation with MAE, MSE, RMSE
    print('MAE:{}'.format(mean_absolute_error(y_test,predictions)))
    print('MSE:{}'.format(mean_squared_error(y_test,predictions)))
    print('RMSE:{}'.format(np.sqrt(mean_squared_error(y_test,predictions))))
    
    sns.distplot(y_test-predictions)

In [73]:
pip install catboost

In [74]:
from sklearn.linear_model import  Ridge, Lasso, LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,  AdaBoostRegressor, BaggingRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

In [75]:
predict(Ridge())

In [76]:
predict(Lasso())

In [77]:
predict(LinearRegression())

In [78]:
predict(DecisionTreeRegressor())

In [79]:
predict(RandomForestRegressor())

In [80]:
predict(GradientBoostingRegressor())

In [81]:
pip install lightgbm

In [82]:
predict(LGBMRegressor())

In [83]:
predict(XGBRegressor())

In [84]:
predict(XGBRFRegressor())

In [85]:
predict(KNeighborsRegressor())

In [86]:
predict(GaussianNB())

In [87]:
predict(SVR())

In [88]:
predict(AdaBoostRegressor())

In [89]:
predict(BaggingRegressor())

In [90]:
predict(SGDRegressor())

In [91]:
predict(CatBoostRegressor())

### Hyperparameter Tuning

In [92]:
from catboost import CatBoostRegressor
catboost = CatBoostRegressor(learning_rate=0.1,random_strength=100)
catboost.fit(X_train, y_train)
predictions = catboost.predict(X_test)
r2score=r2_score(y_test,predictions)
print("r2 score: {}".format(r2score))

In [93]:
test_preds = catboost.predict(X_test)
test_preds

In [94]:
test_probs = catboost.predict(X_test)
test_probs

In [95]:
y_pred = catboost.predict(X_test)

In [96]:
pred_y = pd.DataFrame({'actual_value': y_test, 'predicted_value': y_pred, 'Difference': y_test-y_pred})
pred_y[:20]

In [97]:
plt.scatter(y_test, y_pred, alpha = 0.5)
plt.xlabel("y_test")
plt.ylabel("y_pred")
plt.show()

In [98]:
#feature importance
imp_df = pd.DataFrame({
    'feature': X_train.columns,
    'imp': catboost.feature_importances_
}).sort_values('imp', ascending=False)

In [99]:
imp_df.head(10)

In [100]:
import matplotlib.pyplot as plt

plt.title('Feature Importance')
sns.barplot(data=imp_df.head(10), x='imp', y='feature');

In [101]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, predictions)

In [102]:
#input

def predict_input(single_input):
    input_df = pd.DataFrame([single_input])
    input_df[numeric_cols] = scaler.transform(input_df[numeric_cols])
    input_df[encoded_cols] = encoder.transform(input_df[categorical_cols])
    X_input = input_df[numeric_cols + encoded_cols]
    pred = catboost.predict(X_input)[0]
    
    return pred

In [103]:
input_new = {'Airline': 'IndiGo',
             'Source': 'Hyderabad',
             'Destination': 'New Delhi',
             'Total_Stops': 'non-stop',
             'journey_day': 31,
             'journey_month': 12,
             'Dep_Time_hour': 5,
             'Dep_Time_min' : 15,
             'Arrival_Time_hour' : 7,
             'Arrival_Time_min' : 20,
            }

In [104]:
predict_input(input_new)

## Reference

1. https://www.python.org/dev/peps/pep-0008/
2. https://www.youtube.com/watch?v=wJQIGXSq504
3. https://www.ijert.org/a-survey-on-flight-pricing-prediction-using-machine-learning
4. S.B. Kotsiantis, Decision trees: a recent overview, Artificial Intelligence Review, vol. 39, no. 4, pp. 261-283, 2013.
5. L. Breiman, Random forests, Machine Learning, vol. 45, pp. 5-32, 2001.
6. https://www.youtube.com/watch?v=y4EMEpEnElQ